In [1]:
import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import time

In [2]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'LXnMWcYQhipLAS7rImEzmZ3CkrU033FMha9cwVSngG4vbufTsAOCQQ=='

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/1.exhibitor.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：1



,No,Name,Presentation,Category,Key Word,Url,Homepage,Last Approval Date,Json_Social,IsTopCompany,...,【Addressrdm】WritingSystem,【Address】City,【Address】Country,【Address】Email,【Address】Fax,【Address】POBCity,【Address】POBZip,【Address】Street,【Address】Tel,【Address】Zip
0,6330,Taiwan Brake Technology Crop.,2.1 K95,1.1 驾驶系统（发动机、变速箱、动力总成、排气装置）\n1.2 底盘部分（车桥、方向盘、制...,1. BRAKE PADS\n2. BRAKE SHOES\n3. INTIMA\n4. R...,https://automechanika-shanghai.hk.messefrankfu...,www.taiwanbrake.com,2024-11-07T11:28:51,,,...,HANS,Tainan,台湾,tbt261@taiwanbrake.com,+886 6 593 3528,,,"No. 125-2, Zhonglun, Anding Dist.,",+886 6 593 3800,745002


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                                            6330
 Name                                                 Taiwan Brake Technology Crop.
 Presentation                                                               2.1 K95
 Category                         1.1 驾驶系统（发动机、变速箱、动力总成、排气装置）\n1.2 底盘部分（车桥、方向盘、制...
 Key Word                         1. BRAKE PADS\n2. BRAKE SHOES\n3. INTIMA\n4. R...
 Url                              https://automechanika-shanghai.hk.messefrankfu...
 Homepage                                                       www.taiwanbrake.com
 Last Approval Date                                             2024-11-07T11:28:51
 Json_Social                                                                       
 IsTopCompany                                                                      
 Logo                                                                              
 【Addressrdm】Building                                           

In [5]:
crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_request, url_check = f'''https://api.messefrankfurt.com/service/esb_api/exhibitor-service/api/2.1/public/exhibitor/profile/zh-CN/{crawler_series['Url'].removeprefix('https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/').removesuffix('.html')}/AUTOMECHANIKASHANGHAI''', crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(**{'Apikey': api}),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200 or resp.status_code == 401:
            dict_ = resp.json()

            if resp.status_code == 200 and dict_['success']:
                break
            elif resp.status_code == 401 and dict_['status'] == 401 and dict_['message'].strip() == 'API GW: Unauthorized':
                print(f'fuck api {api}')
                time.sleep(3)
                with open('./api.txt', 'r') as api_txt:
                    api = api_txt.read()
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'success': True,
 'fieldErrors': [],
 'systemErrors': [],
 'message': None,
 'result': {'id': 'mf_2_a4wc0a03i15p~mf_2_10000100202401_A4WC0A03I15P_412369',
  'synId': 'mf_2_a4wc0a03i15p',
  'rewriteId': 'taiwan-brake-technology-crop',
  'name': 'Taiwan Brake Technology Crop.',
  'shortDescription': None,
  'exhibition': {'id': 'AUTOMECHANIKASHANGHAI',
   'name': 'Automechanika Shanghai 2024',
   'synonyme': None,
   'exhibitionHall': [{'id': '2.1',
     'name': '2.1',
     'sortKey': '0000000005',
     'nameLabel': {'localId': 'HALLNAME_2.1_AUTOMECHANIKASHANGHAI',
      'labels': {'zh-TW': {'localId': 'HALLNAME_2.1_AUTOMECHANIKASHANGHAI',
        'groupType': 'HALLDATA',
        'text': '2.1',
        'language': 'zh-TW'},
       'zh-CN': {'localId': 'HALLNAME_2.1_AUTOMECHANIKASHANGHAI',
        'groupType': 'HALLDATA',
        'text': '2.1',
        'language': 'zh-CN'},
       'en-GB': {'localId': 'HALLNAME_2.1_AUTOMECHANIKASHANGHAI',
        'groupType': 'HALLDATA',
        'text': 

In [6]:
list_dict_correct_temp = []
if dict_['result']['products']['countTotal'] == 0:
    list_dict_correct_temp.append({'Url': crawler_series['Url']})
else:
    for i, dict_product in enumerate(dict_['result']['products']['products']):
        list_dict_correct_temp.append({'Url': crawler_series['Url'],
                                       '[Product] No': i + 1,
                                       '[Product] Url': f'''{crawler_series['Url'].removesuffix('.html')}/{dict_product['id']}.html'''})

list_dict_correct_temp

[{'Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop.html',
  '[Product] No': 1,
  '[Product] Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop/mf_2_a4wc0a03i15p_4191866_10000100202401.html'},
 {'Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop.html',
  '[Product] No': 2,
  '[Product] Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop/mf_2_a4wc0a03i15p_4191867_10000100202401.html'},
 {'Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop.html',
  '[Product] No': 3,
  '[Product] Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/t

In [7]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop.html',
  '[Product] No': 1,
  '[Product] Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop/mf_2_a4wc0a03i15p_4191866_10000100202401.html'},
 {'Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop.html',
  '[Product] No': 2,
  '[Product] Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop/mf_2_a4wc0a03i15p_4191867_10000100202401.html'},
 {'Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/taiwan-brake-technology-crop.html',
  '[Product] No': 3,
  '[Product] Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/t

In [8]:
crawler_status = True

crawler_status

True

In [9]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                                            6330
 Name                                                 Taiwan Brake Technology Crop.
 Presentation                                                               2.1 K95
 Category                         1.1 驾驶系统（发动机、变速箱、动力总成、排气装置）\n1.2 底盘部分（车桥、方向盘、制...
 Key Word                         1. BRAKE PADS\n2. BRAKE SHOES\n3. INTIMA\n4. R...
 Url                              https://automechanika-shanghai.hk.messefrankfu...
 Homepage                                                       www.taiwanbrake.com
 Last Approval Date                                             2024-11-07T11:28:51
 Json_Social                                                                       
 IsTopCompany                                                                      
 Logo                                                                              
 【Addressrdm】Building                                                       

In [10]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Url', '[Product] No'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_product_url.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,Url,[Product] No,[Product] Url
0,https://automechanika-shanghai.hk.messefrankfu...,1,https://automechanika-shanghai.hk.messefrankfu...
1,https://automechanika-shanghai.hk.messefrankfu...,2,https://automechanika-shanghai.hk.messefrankfu...
2,https://automechanika-shanghai.hk.messefrankfu...,3,https://automechanika-shanghai.hk.messefrankfu...


In [11]:
print('Output Finished !')
print()

Output Finished !



In [12]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_product_url_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Name,Presentation,Category,Key Word,Url,Homepage,Last Approval Date,Json_Social,IsTopCompany,...,【Address】City,【Address】Country,【Address】Email,【Address】Fax,【Address】POBCity,【Address】POBZip,【Address】Street,【Address】Tel,【Address】Zip,Check_Url
0,6330,Taiwan Brake Technology Crop.,2.1 K95,1.1 驾驶系统（发动机、变速箱、动力总成、排气装置）\n1.2 底盘部分（车桥、方向盘、制...,1. BRAKE PADS\n2. BRAKE SHOES\n3. INTIMA\n4. R...,https://automechanika-shanghai.hk.messefrankfu...,www.taiwanbrake.com,2024-11-07T11:28:51,,,...,Tainan,台湾,tbt261@taiwanbrake.com,+886 6 593 3528,,,"No. 125-2, Zhonglun, Anding Dist.,",+886 6 593 3800,745002,https://automechanika-shanghai.hk.messefrankfu...


In [13]:
print('Done ~')

Done ~
